In [4]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

# Загрузка данных
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

# Уникальные годы для слайдеров
years = sorted(df['year'].unique())

app.layout = html.Div([
    html.H1(children='Визуализация данных Gapminder', style={'textAlign':'center'}),
    
    dcc.Tabs([
        # Вкладка 1: Линейный график с множественным выбором стран
        dcc.Tab(label='Линейный график', children=[
            html.Div([
                html.H3('Сравнение стран'),
                html.Label('Выберите страны:'),
                dcc.Dropdown(
                    id='line-country-selection',
                    options=[{'label': country, 'value': country} for country in sorted(df['country'].unique())],
                    value=['Canada'],
                    multi=True
                ),
                html.Label('Выберите показатель для оси Y:'),
                dcc.Dropdown(
                    id='line-y-axis-selection',
                    options=[
                        {'label': 'Продолжительность жизни', 'value': 'lifeExp'},
                        {'label': 'Население', 'value': 'pop'},
                        {'label': 'ВВП на душу населения', 'value': 'gdpPercap'}
                    ],
                    value='pop'
                ),
                dcc.Graph(id='line-chart')
            ])
        ]),
        
        # Вкладка 2: Пузырьковая диаграмма
        dcc.Tab(label='Пузырьковая диаграмма', children=[
            html.Div([
                html.H3('Пузырьковая диаграмма'),
                html.Div([
                    html.Div([
                        html.Label('Ось X:'),
                        dcc.Dropdown(
                            id='bubble-x-axis',
                            options=[
                                {'label': 'Продолжительность жизни', 'value': 'lifeExp'},
                                {'label': 'Население', 'value': 'pop'},
                                {'label': 'ВВП на душу населения', 'value': 'gdpPercap'}
                            ],
                            value='gdpPercap'
                        )
                    ], style={'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        html.Label('Ось Y:'),
                        dcc.Dropdown(
                            id='bubble-y-axis',
                            options=[
                                {'label': 'Продолжительность жизни', 'value': 'lifeExp'},
                                {'label': 'Население', 'value': 'pop'},
                                {'label': 'ВВП на душу населения', 'value': 'gdpPercap'}
                            ],
                            value='lifeExp'
                        )
                    ], style={'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        html.Label('Размер пузырька:'),
                        dcc.Dropdown(
                            id='bubble-size',
                            options=[
                                {'label': 'Продолжительность жизни', 'value': 'lifeExp'},
                                {'label': 'Население', 'value': 'pop'},
                                {'label': 'ВВП на душу населения', 'value': 'gdpPercap'}
                            ],
                            value='pop'
                        )
                    ], style={'width': '33%', 'display': 'inline-block'})
                ]),
                html.Label('Выберите год:'),
                dcc.Slider(
                    id='year-slider',
                    min=min(years),
                    max=max(years),
                    value=max(years),
                    marks={str(year): str(year) for year in years},
                    step=None
                ),
                dcc.Graph(id='bubble-chart')
            ])
        ]),
        
        # Вкладка 3: Топ-15 стран
        dcc.Tab(label='Топ-15 стран по населению', children=[
            html.Div([
                html.H3('Топ-15 стран по населению'),
                html.Label('Выберите год:'),
                dcc.Slider(
                    id='top15-year-slider',
                    min=min(years),
                    max=max(years),
                    value=max(years),
                    marks={str(year): str(year) for year in years},
                    step=None
                ),
                dcc.Graph(id='top15-chart')
            ])
        ]),
        
        # Вкладка 4: Круговая диаграмма
        dcc.Tab(label='Население по континентам', children=[
            html.Div([
                html.H3('Распределение населения по континентам'),
                html.Label('Выберите год:'),
                dcc.Slider(
                    id='pie-year-slider',
                    min=min(years),
                    max=max(years),
                    value=max(years),
                    marks={str(year): str(year) for year in years},
                    step=None
                ),
                dcc.Graph(id='continent-pie-chart')
            ])
        ])
    ])
])

# Callback для линейного графика
@callback(
    Output('line-chart', 'figure'),
    [Input('line-country-selection', 'value'),
     Input('line-y-axis-selection', 'value')]
)
def update_line_chart(countries, y_axis):
    if not countries:
        return px.line()
    
    filtered_df = df[df['country'].isin(countries)]
    
    # Получаем названия для оси Y
    y_labels = {
        'lifeExp': 'Продолжительность жизни',
        'pop': 'Население',
        'gdpPercap': 'ВВП на душу населения'
    }
    
    fig = px.line(filtered_df, x='year', y=y_axis, color='country',
                 labels={y_axis: y_labels.get(y_axis, y_axis), 'year': 'Год'})
    return fig

# Callback для пузырьковой диаграммы
@callback(
    Output('bubble-chart', 'figure'),
    [Input('bubble-x-axis', 'value'),
     Input('bubble-y-axis', 'value'),
     Input('bubble-size', 'value'),
     Input('year-slider', 'value')]
)
def update_bubble_chart(x_axis, y_axis, size, year):
    filtered_df = df[df['year'] == year]
    
    # Названия для осей
    axis_labels = {
        'lifeExp': 'Продолжительность жизни',
        'pop': 'Население',
        'gdpPercap': 'ВВП на душу населения'
    }
    
    fig = px.scatter(filtered_df, x=x_axis, y=y_axis, size=size,
                    color='continent', hover_name='country',
                    log_x=True if x_axis == 'gdpPercap' else False,
                    labels={
                        x_axis: axis_labels.get(x_axis, x_axis),
                        y_axis: axis_labels.get(y_axis, y_axis),
                        size: axis_labels.get(size, size),
                        'continent': 'Континент'
                    },
                    title=f'Данные за {year} год')
    return fig

# Callback для топ-15 стран
@callback(
    Output('top15-chart', 'figure'),
    [Input('top15-year-slider', 'value')]
)
def update_top15_chart(year):
    filtered_df = df[df['year'] == year]
    top15 = filtered_df.sort_values('pop', ascending=False).head(15)
    fig = px.bar(top15, x='country', y='pop', color='continent',
                labels={'pop': 'Население', 'country': 'Страна', 'continent': 'Континент'},
                title=f'Топ-15 стран по населению в {year} году')
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    return fig

# Callback для круговой диаграммы
@callback(
    Output('continent-pie-chart', 'figure'),
    [Input('pie-year-slider', 'value')]
)
def update_pie_chart(year):
    filtered_df = df[df['year'] == year]
    continent_pop = filtered_df.groupby('continent')['pop'].sum().reset_index()
    fig = px.pie(continent_pop, values='pop', names='continent',
                title=f'Распределение населения по континентам в {year} году',
                labels={'continent': 'Континент', 'pop': 'Население'})
    fig.update_traces(textinfo='percent+label')
    return fig

if __name__ == '__main__':
    app.run(debug=True)